In [7]:
import os
import gradio as gr
from PIL import Image
import numpy as np
import cv2
from tensorflow.keras.models import load_model

In [8]:
model = load_model('custom_unet.h5')

In [38]:
"""
Blur around human:
The function preprocesses the image in the correct dimensions that the neural network is compatible with. The model predicts the mask and then resizes the mask for the original image resolution.
It inverts the mask capturing the surroundings of the human. Uses OpenCV to apply a blurred effect on the inverted mask area then replicate the mask across three channels to make it compatible with rgb.

"""
def predict_segmentation(input_image):
    image_np = np.array(input_image)
    image_resized = cv2.resize(image_np, (256, 256)) / 255.0
    image_expanded = np.expand_dims(image_resized, axis=0)

    predicted_mask = model.predict(image_expanded)[0]
    predicted_mask = (predicted_mask > 0.5).astype(np.uint8)

    mask_resized = cv2.resize(predicted_mask, (input_image.size[0], input_image.size[1]))
    inverted_mask = 1 - mask_resized

    blurred_background = cv2.GaussianBlur(image_np, (51, 51), 0)

    inverted_mask_3ch = np.stack((inverted_mask,)*3, axis=-1)

    segmented_image = np.where(inverted_mask_3ch == 1, blurred_background, image_np)

    segmented_pil = Image.fromarray(segmented_image.astype('uint8'), 'RGB')
    return segmented_pil

In [ ]:
# Minimal gradio code for bringing model to a webapp.
demo = gr.Interface(
    predict_segmentation,
    gr.Image(type="pil"),
    "image",
)

if __name__ == "__main__":
    demo.launch()